In [22]:
import pandas as pd
import numpy as np

In [17]:
# df = pd.read_csv('data_processed.csv',encoding = 'latin-1',lineterminator='\n')

In [18]:
from sklearn.model_selection import train_test_split
cols = [i for i in df.columns if i != 'hammer_price']
X = df[cols]
Y = df.hammer_price

print(X[0:5])
print(Y[0:5])

# Split our data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.20, random_state=1)

   artist_birth_year  artist_death_year          artist_name  \
0               1898             1986.0          Henry Moore   
1               1906             1969.0      Serge Poliakoff   
2               1931             2004.0       Tom Wesselmann   
3               1928             2011.0  Helen Frankenthaler   
4               1931             2004.0       Tom Wesselmann   

  artist_nationality              auction_date                 category  \
0            British  1989-09-08T00:00:00.000Z     Other works on paper   
1            Russian  1998-10-29T01:00:00.000Z                 Painting   
2           American  2000-02-23T01:00:00.000Z  (Screen)Print & Gravure   
3           American  2002-10-14T00:00:00.000Z                 Painting   
4           American  2002-10-22T00:00:00.000Z                 Painting   

  currency      edition  estimate_high  estimate_low       location  \
0      USD  Henry Moore       9.800437      9.474941         Boston   
1      USD  Henry Moor

In [19]:

# from sklearn.metrics import mean_squared_error

# >>> y_true = [3, -0.5, 2, 7]
# >>> y_pred = [2.5, 0.0, 2, 8]
# >>> mean_squared_error(y_true, y_pred)
# 0.375



In [20]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [21]:
model_ridge = Ridge()
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "CV root mean squared error vs Alpha ")
plt.xlabel("alpha")
plt.ylabel("rmse")
cv_ridge.min()

ValueError: could not convert string to float: 'Henry Moore'

In [ ]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, Y_train)
rmse_cv(model_lasso)
coef = pd.Series(model_lasso.coef_, index = X_train.columns)

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

In [ ]:
## why do we look at residuals????
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")


In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [ ]:
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
model_xgb.fit(X_train, y)

In [ ]:
xgb_preds = np.expm1(model_xgb.predict(X_test))
lasso_preds = np.expm1(model_lasso.predict(X_test))

In [ ]:
predictions = pd.DataFrame({"xgb":xgb_preds, "lasso":lasso_preds})
predictions.plot(x = "xgb", y = "lasso", kind = "scatter")

In [ ]:
preds = 0.7*lasso_preds + 0.3*xgb_preds
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
X_train = StandardScaler().fit_transform(X_train)

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, random_state = 3)

In [ ]:
X_tr.shape

In [ ]:
X_tr

In [ ]:
model = Sequential()
#model.add(Dense(256, activation="relu", input_dim = X_train.shape[1]))
model.add(Dense(1, input_dim = X_train.shape[1], W_regularizer=l1(0.001)))

model.compile(loss = "mse", optimizer = "adam")

In [ ]:
model.summary()

In [ ]:
hist = model.fit(X_tr, y_tr, validation_data = (X_val, y_val))

In [ ]:
pd.Series(model.predict(X_val)[:,0]).hist()